In [ ]:
import requests
from datetime import datetime, timezone, timedelta
import os
import json

from handle_base64 import *

# --- Cấu hình ---
# Lưu ý: API Gemini này không dùng thư viện OpenAI, nên không cần biến client = OpenAI(...)
GEMINI_API_URL = "https://api.thucchien.ai/gemini/v1beta/models/gemini-2.5-flash-image-preview:generateContent"
# SỬ DỤNG AI_API_KEY TỪ CODE GỐC LÀM GEMINI KEY
# Lưu ý: API này dùng header 'x-goog-api-key' thay vì 'Authorization: Bearer'
AI_API_KEY = "sk-YzYUqhbM9WskmD2AQDUXJQ" 
MODEL_NAME = "gemini-2.5-flash-image-preview"

# --- Cấu hình log & thư mục ---
LOG_FILE_SUMMARY = "logs/summary_gemini.log" # Đổi tên log để tránh ghi đè
IMAGE_DIR = "results_images_gemini"        # thư mục lưu ảnh
os.makedirs(IMAGE_DIR, exist_ok=True)
os.makedirs("logs", exist_ok=True)

# --- Cấu hình prompt & payload (body) API ---
prompt = "remove background."

# Payload (body) tương ứng với cURL bạn cung cấp
# payload = {
#     "contents": [{
#         "parts": [{
#             "text": prompt,
#             "mime_type" : "image/png",
#             "data" : encode_image_to_base64("C:/Users/Admin/Desktop/Codice_Vincente/notebook/video/inputs/image-Photoroom222-(1).png")
#         }]
#     }],
#     "generationConfig": {
#         "imageConfig": {
#             "aspectRatio": "9:16"
#         }
#     }
# }
payload = {
    "contents": [{
        "parts": [
            {
                "text": prompt
            },
            {
                "inline_data": {  # hoặc inlineData tùy endpoint (ở đây dùng inline_data cho API thucchien)
                    "mime_type": "image/jpeg",
                    "data": encode_image_to_base64(
                        "C:/Users/Admin/Desktop/Codice_Vincente/Leopard_2_A5_der_Bundeswehr.jpeg"
                    )
                }
            }
        ]
    }],
    "generationConfig": {
        "imageConfig": {
            "aspectRatio": "16:9"
        }
    }
}


# --- Thiết lập múi giờ Hà Nội (UTC+7) ---
hanoi_tz = timezone(timedelta(hours=7))
timestamp = datetime.now(hanoi_tz).strftime("%Y-%m-%d %H:%M:%S")
timestamp_safe = datetime.now(hanoi_tz).strftime("%Y%m%d_%H%M%S")

# --- Gọi API ---
print(f"🛠️ Đang gọi API tạo ảnh với model: {MODEL_NAME}...")

headers = {
    'x-goog-api-key': AI_API_KEY,
    'Content-Type': 'application/json'
}

try:
    response = requests.post(GEMINI_API_URL, headers=headers, json=payload)
    print("🔍 Response status:", response.status_code)
    print("🔍 Response text:", response.text)
    response.raise_for_status() # Báo lỗi nếu status code là 4xx hoặc 5xx
    api_response = response.json()
except requests.exceptions.RequestException as e:
    print(f"❌ Lỗi khi gọi API: {e}")
    # Có thể ghi log lỗi chi tiết hơn ở đây nếu cần
    exit()

# --- Xử lý và Lưu ảnh và ghi log ---
saved_images = []

# Kết quả ảnh nằm trong response['candidates'][0]['content']['parts']
image_parts = api_response.get('candidates', [{}])[0].get('content', {}).get('parts', [])

for i, part in enumerate(image_parts):
    # Kiểm tra xem part có chứa ảnh base64 không
    if 'inlineData' in part and part['inlineData']['mimeType'].startswith('image/'):
        b64_data = part['inlineData']['data']
        # Giải mã base64
        image_data = part['inlineData']['data']
        
        # Tạo tên file theo timestamp
        # Gemini trả về ảnh là JPEG (image/jpeg), nên đổi đuôi là .jpeg
        image_filename = f"image_{timestamp_safe}_{i+1}.jpeg" 
        save_path = os.path.join(IMAGE_DIR, image_filename)
        
        # Lưu ảnh
        with open(save_path, 'wb') as f:
            # Dữ liệu base64 trong response đã sẵn sàng để giải mã
            f.write(base64.b64decode(b64_data)) 
        
        print(f"✅ Ảnh {i+1} đã lưu tại: {save_path}")
        saved_images.append(save_path)

if not saved_images:
    print("⚠️ API trả về thành công nhưng không tìm thấy dữ liệu ảnh để lưu.")

# --- Tạo log entry ---
log_entry = {
    "timestamp": timestamp,
    "prompt": prompt,
    "model": MODEL_NAME,
    "aspectRatio": payload['generationConfig']['imageConfig']['aspectRatio'],
    "saved_images": saved_images,
    "full_response": api_response
}

# --- Ghi log tóm tắt (summary đẹp) ---
with open(LOG_FILE_SUMMARY, "a", encoding="utf-8") as f:
    f.write(f"\n=================== 🕒 {timestamp} (UTC+7) ===================\n\n")
    f.write("🎨 Prompt:\n")
    f.write(prompt + "\n\n")
    f.write(f"📊 Model: {MODEL_NAME}\n")
    f.write(f"📐 Aspect Ratio: {payload['generationConfig']['imageConfig']['aspectRatio']}\n")
    f.write(f"🖼️ Ảnh đã lưu ({len(saved_images)}):\n")
    for path in saved_images:
        f.write(f"   → {path}\n")
    f.write("\n======================================================================\n")

print(f"📝 Đã ghi log tóm tắt vào {LOG_FILE_SUMMARY}")